<a href="https://colab.research.google.com/github/fegastal/PosTech-DataAnalytics_TechChallenge/blob/main/PNAD_Covid19_RM_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install basedosdados

In [ ]:
import pandas as pd
import basedosdados as bd ## database management system for Bra# zilian public data
import numpy as np
import seaborn as sns
import os
from matplotlib import pyplot
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType
from pyspark.sql.functions import col, expr, concat, when

spark = SparkSession.builder.appName("PNAD_Covic19").getOrCreate()
spark

In [ ]:
# dict_basedosdados = bd.read_table(dataset_id='br_ibge_pnad_covid',
# table_id='dicionario',
# billing_project_id="basedosdados-1")

In [ ]:
# base_df = bd.read_table(dataset_id='br_ibge_pnad_covid',
#   table_id='microdados',
#   billing_project_id="basedosdados-1") \
#           .fillna(0)

In [ ]:
# base_df_sql = bd.read_sql(query="SELECT  * \
#                                   FROM `basedosdados.br_ibge_pnad_covid.microdados` cov_md \
#                                   WHERE cov_md.mes in (9,10,11)") \
#                                 .fillna(0)


In [ ]:
# base_df_sql

In [ ]:
# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [ ]:
# covid_df = spark.createDataFrame(data = base_df_sql)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
df = spark.read.csv('/content/gdrive/MyDrive/colab_notebooks/pnad_covid19.csv',inferSchema = True,header=True)

In [ ]:
# covid_df.collect()

In [ ]:
df.printSchema()

root
 |-- Ano: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- CAPITAL: string (nullable = true)
 |-- RM_RIDE: string (nullable = true)
 |-- V1008: string (nullable = true)
 |-- V1012: string (nullable = true)
 |-- V1013: string (nullable = true)
 |-- V1016: string (nullable = true)
 |-- Estrato: string (nullable = true)
 |-- UPA: string (nullable = true)
 |-- V1022: string (nullable = true)
 |-- V1023: string (nullable = true)
 |-- V1030: string (nullable = true)
 |-- V1031: string (nullable = true)
 |-- V1032: string (nullable = true)
 |-- posest: string (nullable = true)
 |-- A001: string (nullable = true)
 |-- A001A: string (nullable = true)
 |-- A001B1: string (nullable = true)
 |-- A001B2: string (nullable = true)
 |-- A001B3: string (nullable = true)
 |-- A002: string (nullable = true)
 |-- A003: string (nullable = true)
 |-- A004: string (nullable = true)
 |-- A005: string (nullable = true)
 |-- A006: string (nullable = true)
 |-- A007: string (nullable = true)


In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

Selecting specific columns from data dictionary

In [ ]:
df = df.select(['UF','V1022','A002','A003','B0011','B0012','B0013','B0015','B0018',
                'B0019','B00111','B002','B005','B007','C001','C007D','C007E','C01011','C013','D0051'
           ])

In [ ]:
df.show()

+---+-----+----+----+-----+-----+-----+-----+-----+-----+------+----+----+----+----+-----+-----+------+----+-----+
| UF|V1022|A002|A003|B0011|B0012|B0013|B0015|B0018|B0019|B00111|B002|B005|B007|C001|C007D|C007E|C01011|C013|D0051|
+---+-----+----+----+-----+-----+-----+-----+-----+-----+------+----+----+----+----+-----+-----+------+----+-----+
| 11|    1| 036|   1|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   1|   1|   06| NULL|    04|NULL|    1|
| 11|    1| 030|   2|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   2|   1|   20| NULL|    04|NULL|    1|
| 11|    1| 013|   1|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   1|NULL| NULL| NULL|  NULL|NULL|    1|
| 11|    1| 011|   1|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   1|NULL| NULL| NULL|  NULL|NULL|    1|
| 11|    1| 057|   2|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   2|   2| NULL| NULL|  NULL|NULL|    2|
| 11|    1| 058|   2|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   2| 

In [ ]:
# importing basedosdados dictionary for the table
df_dict = spark.createDataFrame(
    bd.read_table(dataset_id='br_ibge_pnad_covid',
    table_id='dicionario',
    billing_project_id="basedosdados-1").fillna(0)
)

Downloading: 100%|██████████| 554/554 [00:00<00:00, 2578.19rows/s]


In [ ]:
df_dict = df_dict.withColumn('chave',col('chave').cast('int')). \
                            fillna(0)

In [ ]:
df_dict.show()

+----------+-----------+-----+------------------+--------------------+
| id_tabela|nome_coluna|chave|cobertura_temporal|               valor|
+----------+-----------+-----+------------------+--------------------+
|microdados|    capital|   11| 2020-05(1)2020-11|Município de Port...|
|microdados|    capital|   12| 2020-05(1)2020-11|Município de Rio ...|
|microdados|    capital|   13| 2020-05(1)2020-11|Município de Mana...|
|microdados|    capital|   14| 2020-05(1)2020-11|Município de Boa ...|
|microdados|    capital|   15| 2020-05(1)2020-11|Município de Belé...|
|microdados|    capital|   16| 2020-05(1)2020-11|Município de Maca...|
|microdados|    capital|   17| 2020-05(1)2020-11|Município de Palm...|
|microdados|    capital|   21| 2020-05(1)2020-11|Município de São ...|
|microdados|    capital|   22| 2020-05(1)2020-11|Município de Tere...|
|microdados|    capital|   23| 2020-05(1)2020-11|Município de Fort...|
|microdados|    capital|   24| 2020-05(1)2020-11|Município de Nata...|
|micro

In [ ]:
df_dict = df_dict.withColumn("chave_coluna", concat(col("nome_coluna"), col("chave")))

In [ ]:
df_dict.show()

+----------+-----------+-----+------------------+--------------------+------------+
| id_tabela|nome_coluna|chave|cobertura_temporal|               valor|chave_coluna|
+----------+-----------+-----+------------------+--------------------+------------+
|microdados|    capital|   11| 2020-05(1)2020-11|Município de Port...|   capital11|
|microdados|    capital|   12| 2020-05(1)2020-11|Município de Rio ...|   capital12|
|microdados|    capital|   13| 2020-05(1)2020-11|Município de Mana...|   capital13|
|microdados|    capital|   14| 2020-05(1)2020-11|Município de Boa ...|   capital14|
|microdados|    capital|   15| 2020-05(1)2020-11|Município de Belé...|   capital15|
|microdados|    capital|   16| 2020-05(1)2020-11|Município de Maca...|   capital16|
|microdados|    capital|   17| 2020-05(1)2020-11|Município de Palm...|   capital17|
|microdados|    capital|   21| 2020-05(1)2020-11|Município de São ...|   capital21|
|microdados|    capital|   22| 2020-05(1)2020-11|Município de Tere...|   cap

In [ ]:
# for column_name in df.columns:
#     df = df.withColumnRenamed(column_name, column_name.lower())

In [ ]:
from pyspark.sql.functions import col
for column_name in df.columns:
  df = df.withColumn(column_name, col(column_name).cast('int'))

In [ ]:
df.show()

+---+-----+----+----+-----+-----+-----+-----+-----+-----+------+----+----+----+----+-----+-----+------+----+-----+
| UF|V1022|A002|A003|B0011|B0012|B0013|B0015|B0018|B0019|B00111|B002|B005|B007|C001|C007D|C007E|C01011|C013|D0051|
+---+-----+----+----+-----+-----+-----+-----+-----+-----+------+----+----+----+----+-----+-----+------+----+-----+
| 11|    1|  36|   1|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   1|   1|    6| NULL|     4|NULL|    1|
| 11|    1|  30|   2|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   2|   1|   20| NULL|     4|NULL|    1|
| 11|    1|  13|   1|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   1|NULL| NULL| NULL|  NULL|NULL|    1|
| 11|    1|  11|   1|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   1|NULL| NULL| NULL|  NULL|NULL|    1|
| 11|    1|  57|   2|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   2|   2| NULL| NULL|  NULL|NULL|    2|
| 11|    1|  58|   2|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|   2| 

In [ ]:
df = df.fillna(0)
df.show()

+---+-----+----+----+-----+-----+-----+-----+-----+-----+------+----+----+----+----+-----+-----+------+----+-----+
| UF|V1022|A002|A003|B0011|B0012|B0013|B0015|B0018|B0019|B00111|B002|B005|B007|C001|C007D|C007E|C01011|C013|D0051|
+---+-----+----+----+-----+-----+-----+-----+-----+-----+------+----+----+----+----+-----+-----+------+----+-----+
| 11|    1|  36|   1|    2|    2|    2|    2|    2|    2|     2|   0|   0|   1|   1|    6|    0|     4|   0|    1|
| 11|    1|  30|   2|    2|    2|    2|    2|    2|    2|     2|   0|   0|   2|   1|   20|    0|     4|   0|    1|
| 11|    1|  13|   1|    2|    2|    2|    2|    2|    2|     2|   0|   0|   1|   0|    0|    0|     0|   0|    1|
| 11|    1|  11|   1|    2|    2|    2|    2|    2|    2|     2|   0|   0|   1|   0|    0|    0|     0|   0|    1|
| 11|    1|  57|   2|    2|    2|    2|    2|    2|    2|     2|   0|   0|   2|   2|    0|    0|     0|   0|    2|
| 11|    1|  58|   2|    2|    2|    2|    2|    2|    2|     2|   0|   0|   2| 

In [ ]:
df.printSchema()

root
 |-- UF: integer (nullable = true)
 |-- V1022: integer (nullable = true)
 |-- A002: integer (nullable = true)
 |-- A003: integer (nullable = true)
 |-- B0011: integer (nullable = true)
 |-- B0012: integer (nullable = true)
 |-- B0013: integer (nullable = true)
 |-- B0015: integer (nullable = true)
 |-- B0018: integer (nullable = true)
 |-- B0019: integer (nullable = true)
 |-- B00111: integer (nullable = true)
 |-- B002: integer (nullable = true)
 |-- B005: integer (nullable = true)
 |-- B007: integer (nullable = true)
 |-- C001: integer (nullable = true)
 |-- C007D: integer (nullable = true)
 |-- C007E: integer (nullable = true)
 |-- C01011: integer (nullable = true)
 |-- C013: integer (nullable = true)
 |-- D0051: integer (nullable = true)



## Altering data and column names

In [ ]:
new_column_name_dict = {
    "V1022": "housing_status",
    "A002": "age",
    "A003": "sex",
    "B0011": "had_fever_last_week",
    "B0012": "had_cough_last_week",
    "B0013": "had_sore_throat_last_week",
    "B0015": "had_headache_last_week",
    "B0018": "had_runny_nose_last_week",
    "B0019": "had_lethargy_last_week",
    "B00111": "had_loss_of_smell_last_week",
    "B002": "went_to_healthcare_facility",
    "B005": "was_hospitalized",
    "B007": "has_health_insurance",
    "C001": "worked_last_week",
    "C007D": "main_job_activity",
    "C007E": "how_many_employees",
    "C01011": "income_range",
    "C013": "has_worked_remote"
}

In [ ]:
for old_col, new_col in new_column_name_dict.items():
    df = df.withColumnRenamed(old_col, new_col)

In [ ]:
df.show()

+---+--------------+---+---+-------------------+-------------------+-------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+--------------------+----------------+-----------------+------------------+------------+-----------------+-----+
| UF|housing_status|age|sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|has_health_insurance|worked_last_week|main_job_activity|how_many_employees|income_range|has_worked_remote|D0051|
+---+--------------+---+---+-------------------+-------------------+-------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+--------------------+----------------+-----------------+----

In [ ]:
symptoms_dict = {
    1: "Yes",
    2: "No",
    3: "Don't know",
    9: "Ignored",
    0: "Not applicable"
}

In [ ]:
def translate_and_cast(value):
    return symptoms_dict.get(value, None)

In [ ]:
# Columns to translate
symptoms_cols = ['had_fever_last_week','had_cough_last_week',
                 'had_sore_throat_last_week','had_headache_last_week','had_runny_nose_last_week',
                 'had_lethargy_last_week','had_loss_of_smell_last_week','went_to_healthcare_facility','was_hospitalized','has_health_insurance',
                 'worked_last_week','has_worked_remote']

work_cols = ['main_job_activity','how_many_employees','income_range']

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

In [ ]:
translate_udf = udf(translate_and_cast, StringType())

# Apply the UDF to the selected columns
for column in symptoms_cols:
    df = df.withColumn(column, translate_udf(col(column)))

In [ ]:
df.show()

+---+--------------+---+---+-------------------+-------------------+-------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+--------------------+----------------+-----------------+------------------+------------+-----------------+-----+
| UF|housing_status|age|sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|has_health_insurance|worked_last_week|main_job_activity|how_many_employees|income_range|has_worked_remote|D0051|
+---+--------------+---+---+-------------------+-------------------+-------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+--------------------+----------------+-----------------+----

In [ ]:
brazil_states = {
    11: "Rondônia",
    12: "Acre",
    13: "Amazonas",
    14: "Roraima",
    15: "Pará",
    16: "Amapá",
    17: "Tocantins",
    21: "Maranhão",
    22: "Piauí",
    23: "Ceará",
    24: "Rio Grande do Norte",
    25: "Paraíba",
    26: "Pernambuco",
    27: "Alagoas",
    28: "Sergipe",
    29: "Bahia",
    31: "Minas Gerais",
    32: "Espírito Santo",
    33: "Rio de Janeiro",
    35: "São Paulo",
    41: "Paraná",
    42: "Santa Catarina",
    43: "Rio Grande do Sul",
    50: "Mato Grosso do Sul",
    51: "Mato Grosso",
    52: "Goiás",
    53: "Distrito Federal"
}

In [ ]:
df = df.withColumn('UF',)

TypeError: ignored